## deltaLens Data Compare Sample

In [1]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from datetime import date

In [2]:
#config

config = load_config("compare.config.json")


In [3]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [8]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42911440,42999117,0,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [12]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,price,42910557,429.0,0.00
1,quantity,42910557,4369.0,0.01
2,trade_type,42910557,21453185.0,50.00
3,symbol,42910557,4291135.0,10.00
4,timestamp,42910557,0.0,0.00


#### [entity]_compare
Entity specific table containing details of comparison

In [13]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,price_left,price_right,price_match,quantity_left,quantity_right,quantity_match,trade_type_left,trade_type_right,trade_type_match,symbol_left,symbol_right,symbol_match,timestamp_left,timestamp_right,timestamp_match,_exists_left,_exists_right,_full_match
0,12042125,757.56,826.51,False,3745,9777,False,BUY,BUY,True,WMT,JPM,False,2024-11-27 21:25:59,2024-06-08 20:35:31,False,True,True,False
1,14395215,488.92,538.41,False,6541,3265,False,SELL,SELL,True,META,JPM,False,2024-12-21 21:26:14,2024-10-26 20:35:48,False,True,True,False
2,21767594,845.22,192.96,False,2187,6797,False,BUY,SELL,False,AMZN,WMT,False,2024-06-12 21:27:01,2024-08-16 20:36:42,False,True,True,False
3,24120271,46.75,315.96,False,8141,8026,False,SELL,BUY,False,AMZN,MSFT,False,2025-01-18 21:27:17,2024-12-05 20:37:00,False,True,True,False
4,1705259,175.34,197.19,False,4009,3961,False,SELL,BUY,False,AMZN,JPM,False,2024-09-20 21:24:52,2024-03-27 20:34:14,False,True,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [15]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

'tradeCompare_2025-02-13.sqlite'